##### Akeem Wells ( ajw3rg@virginia.edu )
##### DS 5001
##### 10 May 2021

# Imports

In [1]:
import os
import pandas
import nltk
import plotly_express as px
from scipy.spatial.distance import pdist

# Config

In [2]:
OHCO = ["song_id","sent_num","token_num"]
gradient_cmap = 'GnBu'
SONGS = OHCO[:1]
SENTS = OHCO[:2]

# Functions

In [3]:
contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "Im": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "wont": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "yall": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "youre": "you are",
    "you've": "you have",
    "wanna": "want to"
}

def replace_contractions(string,contractions= contractions):
    for key in contractions.keys():
        out = string.lower().replace(key,contractions[key])
    return out
    

In [4]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pandas.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pandas.Series(nltk.pos_tag(nltk.word_tokenize(x)))
        return s

    #doc_df.line_str = doc_df.line_str
    df = doc_df.line_str\
        .apply(lambda x: replace_contractions(x))\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    # Add index
    df.index.names = OHCO
    
    return df
def extract_vocabulary(TOKEN):
    try:
        del VOCAB
    except:
        pass
    TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

    VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
        .sort_index().reset_index().rename(columns={'index':'term_str'})
    VOCAB.index.name = 'term_id'
    VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')
    
    # Add Stopwords
    sw = pandas.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
    sw = sw.reset_index().set_index('term_str')
    sw.columns = ['dummy']
    sw.dummy = 1
    VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
    VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
    return VOCAB

def add_stems(VOCAB):
    from nltk.stem.porter import PorterStemmer

    stemmer = PorterStemmer()
    VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)
    return VOCAB

def pre_processing(VOCAB, TOKEN):
    if 'term_rank' not in VOCAB.columns:
        VOCAB = VOCAB.sort_values('n', ascending=False).reset_index()
        VOCAB.index.name = 'term_rank'
        VOCAB = VOCAB.reset_index()
        VOCAB = VOCAB.set_index('term_id')
        VOCAB['term_rank'] = VOCAB['term_rank'] + 1

    TOKEN = TOKEN.dropna()
    VOCAB = VOCAB.dropna()
    TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

    VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).count().iloc[:,0].unstack().idxmax(1)
    return VOCAB, TOKEN

def create_library(list_of_songs, genre):
    data = []
    for row in list_of_songs:
        #print(row)
        title = row[0].split(".txt")[0].split("---")[0].replace("_","").replace(",","").lower()
        artist = row[0].split(".txt")[0].split("---")[1].lower().replace('_', '')
        year = row[2]
        song_file = row[1]
        data.append([title,artist, year,song_file, row[3], genre ])
    # Create the pandas DataFrame
    LIB = pandas.DataFrame(data, columns = ['title', 'artist', 'year','song_file', 'song_id','genre'])
    LIB = LIB.set_index('song_id')
    return LIB

def get_files(rawfiles, genre):
    array = []
    for name in list(rawfiles.keys()):
        if genre in rawfiles[name][1]:
            #print(rawfiles[name][1])
            if genre not in rawfiles[name][1].split("/")[2]: continue
            array.append(rawfiles[name])
    return array

def import_data(dataframe,genre):
    import string
    dataframes = []
    for filename in dataframe['song_file'].tolist():
        #print(filename)
        epub = open(filename, 'r').readlines()

        df = pandas.DataFrame(epub, columns=['line_str'])
        df.line_str = df.line_str.str.strip()
        df['line_str'] = df['line_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['line_str'].str.match(r'^\s*$')]
        
        df['line_str'] = df['line_str'].apply(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
        #df['line_str'] = df['line_str'].apply(lambda s: s.lower())
        df.index.name = 'sent_num'
        #df = df['line_str'].str.split(r"[\s',-]+", expand=True).stack()\
        #.to_frame().rename(columns={0:'token_str'})
        df['title'] = filename.split("/")[-1].split(".txt")[0].split("---")[0].replace("_","").replace(",","").lower()
        df['artist'] = filename.split("/")[-1].split(".txt")[0].split("---")[1].lower().replace('_', '')
        df['year'] = dataframe.loc[dataframe['song_file'] == filename, 'year'].iloc[0]
        df['genre'] = genre
        #df.index.names = ["sent_num","token_num"]
        dataframes.append(df)
    return pandas.concat(dataframes)

In [5]:
#years = ["2015","2016","2017","2018","2019","2020"]
years = [str(x) for x in range(2009,2021)]

In [6]:
# Remove Duplicate Files from Analysis
rawfiles = {}
song_id = 1001
for year in years:
    datapath = "data/{}".format(year)
    for root, dirs, files in os.walk(datapath, topdown=True):
        for name in files:
            #print(os.path.join(root, name))
            #print(name, root)
            if name not in rawfiles.keys():
                if '.DS_Store' in name: continue
                #print((name, os.path.join(root, name), year, str(song_id)))
                rawfiles[name] = (name, os.path.join(root, name), year, str(song_id))
                song_id = song_id + 1

In [7]:
pop_files = get_files(rawfiles, "pop")
country_files = get_files(rawfiles, "country")
rnbhiphop_files = get_files(rawfiles, "rnbhiphop")
rap_files = get_files(rawfiles, "rap")

# Create Library

In [8]:
LIB_POP = create_library(     pop_files,'pop')
LIB_COUNTRY = create_library( country_files,'country')
LIB_RBHH = create_library(    rnbhiphop_files,'rnbhiphop')
LIB_RAP = create_library(     rap_files,'rap')

In [9]:
print("Number of Pop Songs: {}".format(LIB_POP.shape[0]))
print("Number of Country Songs: {}".format(LIB_COUNTRY.shape[0]))
print("Number of R&B / Hip-Hip Songs: {}".format(LIB_RBHH.shape[0]))
print("Number of Rap Songs: {}".format(LIB_RAP.shape[0]))

Number of Pop Songs: 277
Number of Country Songs: 145
Number of R&B / Hip-Hip Songs: 228
Number of Rap Songs: 231


In [10]:
LIB = pandas.concat([LIB_POP, LIB_COUNTRY,LIB_RBHH , LIB_RAP])

In [11]:
LIB.head()

,title,artist,year,song_file,genre
song_id,,,,,
1001,justdance,ladygaga,2009,data/2009/pop/Just_Dance---lady_gaga_.txt,pop
1002,mylifewouldsuckwithoutyou,kellyclarkson,2009,data/2009/pop/My_Life_Would_Suck_Without_You--...,pop
1003,idonothookup,kellyclarkson,2009,data/2009/pop/I_Do_Not_Hook_Up---kelly_clarkso...,pop
1004,paparazzi,ladygaga,2009,data/2009/pop/Paparazzi---lady_gaga.txt,pop
1005,wakingupinvegas,katyperry,2009,data/2009/pop/Waking_Up_In_Vegas---katy_perry.txt,pop


In [12]:
LIB[LIB.artist=='edsheeran']

,title,artist,year,song_file,genre
song_id,,,,,
1303,theateam,edsheeran,2013,data/2013/pop/The_A_Team---ed_sheeran.txt,pop
1371,sing,edsheeran,2014,data/2014/pop/Sing---ed_sheeran.txt,pop
1382,dont,edsheeran,2014,data/2014/pop/Dont---ed_sheeran.txt,pop
1441,photograph,edsheeran,2015,data/2015/pop/Photograph---ed_sheeran.txt,pop
1445,thinkingoutloud,edsheeran,2015,data/2015/pop/Thinking_Out_Loud---ed_sheeran.txt,pop
1577,castleonthehill,edsheeran,2017,data/2017/pop/Castle_On_The_Hill---ed_sheeran.txt,pop
1591,shapeofyou,edsheeran,2017,data/2017/pop/Shape_Of_You---ed_sheeran.txt,pop
1673,perfect,edsheeran,2018,data/2018/pop/Perfect---ed_sheeran.txt,pop
1744,idontcare,edsheeran,2019,data/2019/pop/I_Dont_Care---ed_sheeran_.txt,pop


In [13]:
LIB.shape

(881, 5)

# Imports file into a dataframe

In [14]:
df_pop = import_data(LIB_POP,'pop')
df_country = import_data(LIB_COUNTRY,'country')
df_rbhh = import_data(LIB_RBHH,'rnbhiphop')
df_rap = import_data(LIB_RAP,'rap')

In [15]:
df_out = pandas.concat([df_pop, df_country, df_rbhh, df_rap])

# Drop Duplicate Rows

In [16]:
df_out.shape

(54436, 5)

In [17]:
df_out = df_out.drop_duplicates()

In [18]:
df_out.shape

(34665, 5)

# Create DOC

In [19]:
DOC = df_out.reset_index().merge(LIB.reset_index(),on=['title','artist'] )
DOC = DOC.set_index(['song_id','sent_num'])
columns = ['title','year_x','genre_x','genre_y','year_y','song_file']
DOC.drop(columns, inplace=True, axis=1)

In [20]:
DOC.head()

line_str    artist
song_id sent_num                                                         
1001    0                                                 Truth  ladygaga
        1                                                RedOne  ladygaga
        2                                               Konvict  ladygaga
        3                                          Gaga Ohoh eh  ladygaga
        5         Ive had a little bit too much much Oh oh ohoh  ladygaga

# Create Tokens & Vocab

In [21]:
TOKENS = tokenize(DOC,OHCO=OHCO,ws=True)

### Reduce, Annotate, Add Stems

In [22]:
VOCAB = extract_vocabulary(TOKENS)
VOCAB = add_stems(VOCAB)
VOCAB,TOKENS = pre_processing(VOCAB, TOKENS)

In [23]:
VOCAB = VOCAB.dropna()
TOKENS = TOKENS.dropna()

In [24]:
TOKENS['term_id'] = TOKENS.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

In [25]:
VOCAB['pos_max'] = TOKENS.groupby(['term_id', 'pos']).count().iloc[:,0].unstack().idxmax(1)

#### Checkpoint

In [26]:
TOKENS.head()

pos_tuple pos token_str term_str  term_id
song_id sent_num token_num                                               
1001    0        0            (truth, NN)  NN     truth    truth    11236
        1        0           (redone, NN)  NN    redone   redone     8681
        2        0          (konvict, NN)  NN   konvict  konvict     5937
        3        0             (gaga, NN)  NN      gaga     gaga     4349
                 1             (ohoh, NN)  NN      ohoh     ohoh     7439

In [27]:
VOCAB.head()

,term_rank,term_str,n,num,stop,p_stem,pos_max
term_id,,,,,,,
5343,1,i,11009,0,1,i,NN
12233,2,you,9510,0,1,you,PRP
10790,3,the,8477,0,1,the,DT
11000,4,to,5465,0,1,to,TO
176,5,a,5126,0,1,a,DT


In [28]:
DOC.head()

line_str    artist
song_id sent_num                                                         
1001    0                                                 Truth  ladygaga
        1                                                RedOne  ladygaga
        2                                               Konvict  ladygaga
        3                                          Gaga Ohoh eh  ladygaga
        5         Ive had a little bit too much much Oh oh ohoh  ladygaga

In [29]:
LIB.head()

,title,artist,year,song_file,genre
song_id,,,,,
1001,justdance,ladygaga,2009,data/2009/pop/Just_Dance---lady_gaga_.txt,pop
1002,mylifewouldsuckwithoutyou,kellyclarkson,2009,data/2009/pop/My_Life_Would_Suck_Without_You--...,pop
1003,idonothookup,kellyclarkson,2009,data/2009/pop/I_Do_Not_Hook_Up---kelly_clarkso...,pop
1004,paparazzi,ladygaga,2009,data/2009/pop/Paparazzi---lady_gaga.txt,pop
1005,wakingupinvegas,katyperry,2009,data/2009/pop/Waking_Up_In_Vegas---katy_perry.txt,pop


# Update Tokens table 

In [30]:
TOKENS2 =  TOKENS.reset_index()

In [31]:
TOKENS2 = pandas.merge(TOKENS2, LIB, on = ['song_id'])

In [32]:
TOKENS2 = TOKENS2.set_index(['genre','artist','title','sent_num','token_num'])

In [33]:
TOKENS2

song_id      pos_tuple  \
genre artist            title     sent_num token_num                          
pop   ladygaga          justdance 0        0            1001    (truth, NN)   
                                  1        0            1001   (redone, NN)   
                                  2        0            1001  (konvict, NN)   
                                  3        0            1001     (gaga, NN)   
                                           1            1001     (ohoh, NN)   
...                                                      ...            ...   
rap   megantheestallion savage    58       2            1881      (ooh, NN)   
                                           3            1881      (ooh, NN)   
                                           4            1881      (ooh, NN)   
                                           5            1881      (ooh, NN)   
                                           6            1881     (mwah, NN)   

                                                     pos token_str term_str  \
genre artist            title     sent_num token_num                          
pop   ladygaga          justdance 0        0          NN     truth    truth   
                                  1        0          NN    redone   redone   
                                  2        0          NN   konvict  konvict   
                                  3        0          NN      gaga     gaga   
                                           1          NN      ohoh     ohoh   
...                                                   ..       ...      ...   
rap   megantheestallion savage    58       2          NN       ooh      ooh   
                                           3          NN       ooh      ooh   
                                           4          NN       ooh      ooh   
                                           5          NN       ooh      ooh   
                                           6          NN      mwah     mwah   

                                                      term_id  year  \
genre artist            title     sent_num token_num                  
pop   ladygaga          justdance 0        0            11236  2009   
                                  1        0             8681  2009   
                                  2        0             5937  2009   
                                  3        0             4349  2009   
                                           1             7439  2009   
...                                                       ...   ...   
rap   megantheestallion savage    58       2             7504  2020   
                                           3             7504  2020   
                                           4             7504  2020   
                                           5             7504  2020   
                                           6             7101  2020   

                                                                                           song_file  
genre artist            title     sent_num token_num                                                  
pop   ladygaga          justdance 0        0               data/2009/pop/Just_Dance---lady_gaga_.txt  
                                  1        0               data/2009/pop/Just_Dance---lady_gaga_.txt  
                                  2        0               data/2009/pop/Just_Dance---lady_gaga_.txt  
                                  3        0               data/2009/pop/Just_Dance---lady_gaga_.txt  
                                           1               data/2009/pop/Just_Dance---lady_gaga_.txt  
...                                                                                              ...  
rap   megantheestallion savage    58       2          data/2020/rap/Savage---megan_thee_stallion.txt  
                                           3          data/2020/rap/Savage---megan_thee_stallion.txt  
                                           

In [34]:
columns = ['song_id','year','song_file']
TOKENS2.drop(columns, inplace=True, axis=1)

In [35]:
TOKENS2 = TOKENS2[~TOKENS2.pos.isin(['NNP','NNPS'])]

In [36]:
TOKENS2

pos_tuple pos  \
genre artist            title     sent_num token_num                      
pop   ladygaga          justdance 0        0            (truth, NN)  NN   
                                  1        0           (redone, NN)  NN   
                                  2        0          (konvict, NN)  NN   
                                  3        0             (gaga, NN)  NN   
                                           1             (ohoh, NN)  NN   
...                                                             ...  ..   
rap   megantheestallion savage    58       2              (ooh, NN)  NN   
                                           3              (ooh, NN)  NN   
                                           4              (ooh, NN)  NN   
                                           5              (ooh, NN)  NN   
                                           6             (mwah, NN)  NN   

                                                     token_str term_str  \
genre artist            title     sent_num token_num                      
pop   ladygaga          justdance 0        0             truth    truth   
                                  1        0            redone   redone   
                                  2        0           konvict  konvict   
                                  3        0              gaga     gaga   
                                           1              ohoh     ohoh   
...                                                        ...      ...   
rap   megantheestallion savage    58       2               ooh      ooh   
                                           3               ooh      ooh   
                                           4               ooh      ooh   
                                           5               ooh      ooh   
                                           6              mwah     mwah   

                                                      term_id  
genre artist            title     sent_num token_num           
pop   ladygaga          justdance 0        0            11236  
                                  1        0             8681  
                                  2        0             5937  
                                  3        0             4349  
                                           1             7439  
...                                                       ...  
rap   megantheestallion savage    58       2             7504  
                                           3             7504  
                                           4             7504  
                                           5             7504  
                                           6             7101  

[271956 rows x 5 columns]

# Save

In [37]:
DOC.to_csv('DOC.csv')
LIB.to_csv('LIB.csv')
VOCAB.to_csv('VOCAB.csv')
TOKENS2.to_csv('TOKENS.csv')